# Benchmarks of Strategies for Selecting Outdated Items
This notebook contains the benchmarks related to the selection strategies for context data, which we report in our paper.
Context data are selected from the processed data and included in the next progressive computation step, such that its result approximates that of a _non-progressive_ computation over the processed data.

## Benchmark Configuration

We use the following configuration in our benchmarks:
### Test cases 
- full computation over the entire dataset (upper baseline)
- progressive computation without optimization (lower baseline)
- full computation of processed data
- progressive computation using optimization strategies

### Dataset
- NYC taxis dataset (10 Million items), stored in a compressed CSV file, loaded with DuckDB 

### Variables
- dependent variables: runtime, prediction error
- independent variables: 

## Benchmarks

### Finding an appropriate `chunk size`
The number of items in each chunk dictate the computation time for each chunk in the data, in that the more items we process, the longer the DOI computation takes.
Therefore, the first consideration in our benchmarks is to find the maximum number of items, for which the computation time remains immediate.
Prior work (see Card et al., 1991) has shown this limit to be about one second.

In the cell below, we try different chunk sizes to find the maximum items we can pass to the doi function for computations under 1s.

In [ ]:
from time import time
from setup import *
from test_case import create_test_case

chunk_sizes = [10, 100, 1000, 10000]

for size in chunk_sizes:
  before = time()
  test_case = create_test_case(
    name="__test__", 
    doi=doi,
    c=NoContext(n_dims, None), 
    u=NoUpdate(n_dims, None), 
    s=NoStorage(),
    doi_label=doi_label,
    data_label=data_label,
    data_path=data_path,
    column_data_path=column_data_path,
    id_column=id_column,
    chunk_size=size, 
    update_interval=0,
    path=path,
    max_size=0,
    chunks=1,
    total_size=total_size,
  )
  test_case.run()
  print(f"{size}: {time() - before}s")


### Computing the Baselines
#### Baseline1: Monolithic computation
The ground truth for our strategies is a full computation over the entire dataset without any chunking.
This computation naturally takes a long time to complete, which is why the progressive scenario is so much more effective from a user perspective: we get to see the data much faster.

In the context of the `BenchmarkTestCase` class, the monolithic computation corresponds with running a progression with a single chunk. 

#### Baseline2: Bigger chunks
In addition to the full computation, another important baseline is to compare ourselves against a computation that does not use any strategies, but instead uses the entire `chunk time` to compute a whole new chunk.
The idea here is to compare, whether all the context- and outdated-computations are actually valuable, or whether we could just use all resources on processing new data instead.

In [ ]:
from setup import *
from test_case import run_ground_truth_test_case, run_bigger_chunks_test_case

run_ground_truth_test_case(doi_label, doi, data_label, n_dims, total_size, data_path, column_data_path, id_column, path)
run_bigger_chunks_test_case(doi_label, doi, data_label, n_dims, chunk_size, update_size, context_size, total_size, data_path, column_data_path, id_column, path)

### Running the test cases

In [ ]:
from setup import *
from test_case import create_test_case

total = len(context_strategies) * len(update_strategies)
i = 0
doi_ = doi
print(f"data: {data_label}, {short_test_case_title}\n####")
for c_strat in context_strategies:
  for u_strat in update_strategies:
    i += 1
    print(f"({i}/{total}) context: {c_strat[0]}, update: {u_strat[0]}")

    # check if already completed
    name = f"{c_strat[0]}-{u_strat[0]}"
    if os.path.isfile(f"{path}/doi/{name}.csv") and os.path.isfile(f"{path}/times/{name}.csv"):
      print("skipping test case because already completed.")
      continue
    
    tc = create_test_case(name, data_label, doi_label, data_path, column_data_path, id_column, doi,
                          WindowingStorage(max_size=total_size), c_strat[1](), u_strat[1](), chunk_size,
                          context_size, update_size, update_interval, path, chunks, total_size)
    tc.run()

    print(f"done: {tc.pipeline.total_time}s")

### Evaluation

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import jaccard_score, r2_score

rus = RandomUnderSampler(random_state=0)

def evaluate_test_case(test_case: np.ndarray, ground_truth: np.ndarray):
  # score = jaccard_score(test_case, ground_truth, average="weighted")
  score = r2_score(ground_truth, test_case)
  return score

ground_truth = results_full["doi"]
context_test_case = results_context["doi"]
baseline_test_case = results_chunked["doi"]


evaluate_test_case(baseline_test_case, ground_truth), evaluate_test_case(context_test_case, ground_truth)